In [ ]:
import sys
from pathlib import Path
from os import getcwd, remove
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent.parent))

In [ ]:
import pandas as pd
import numpy as np
from os.path import join

from datetime import datetime, date, timedelta
import random



from bulletin import root, default_input, default_output, agora, hoje, ontem, anteontem, dias_apos, dias_apos_label
from bulletin.services.metabase import Metabase
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.notifica import Notifica
from bulletin.utils.static import Municipios
from bulletin.utils import utils, static
from bulletin.utils.xls_ import fit_cols
from bulletin.utils.normalize import trim_overspace

from bulletin.utils.normalize import normalize_text#normalize_hash, normalize_labels, , date_hash, normalize_number

In [ ]:
from datetime import datetime, date, timedelta

exclusao_pathfile = join(root, 'database', 'casos_confirmados')

today = pd.to_datetime(date.today())
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)
data_retroativos = ontem - timedelta(31)

random.seed(10)

In [ ]:
municipios = Municipios()
municipios['mun_resid'] = municipios['municipio']
municipios.loc[municipios['uf']!='PR','mun_resid'] = municipios.loc[municipios['uf']!='PR','municipio'] + '/' + municipios['uf']

In [ ]:
tabela_base = pd.read_excel('estrategias.xlsx')
tabela_base

In [ ]:
tabela = random.choices(tabela_base['nome'], weights=tabela_base['valor'], k=1000)
tabela = pd.DataFrame(tabela)
tabela.to_excel('teste.xlsx', index=False)

In [ ]:
partial_total = tabela_base['CAIXAS'].sum()
partial_total

In [ ]:
testes_adicionais = random.choices(tabela_base['Municipio'], weights=tabela_base['Populacao'], k=int(n_caixinhas - partial_total))

In [ ]:
tabela_base = tabela_base.join(pd.Series(testes_adicionais, name='CAIXAS_ADICIONAIS').value_counts(), on='Municipio')

In [ ]:
tabela_base['CAIXAS'] = tabela_base['CAIXAS'] + tabela_base['CAIXAS_ADICIONAIS'].fillna(0)
del(tabela_base['CAIXAS_ADICIONAIS'])
tabela_base

In [ ]:
tabela_base['RS'] = tabela_base['RS'].astype(int)
tabela_base['CAIXAS'] = tabela_base['CAIXAS'].astype(int)
tabela_base['TESTES'] = tabela_base['CAIXAS'] * testes_per_caixas
tabela_base.loc['TOTAL', ['Populacao', 'CAIXAS', 'TESTES']] = tabela_base[['Populacao', 'CAIXAS', 'TESTES']].sum()

In [ ]:
tabela_base['Populacao'] = tabela_base['Populacao'].astype(int)
tabela_base['CAIXAS'] = tabela_base['CAIXAS'].astype(int)
tabela_base['TESTES'] = tabela_base['TESTES'].astype(int)
tabela_base

In [ ]:
# WRITER

In [ ]:
writer = pd.ExcelWriter(join(default_output,'testes', filename),
                    engine='xlsxwriter')

In [ ]:
# POR MUNICIPIO

In [ ]:
testes_municipios = tabela_base.rename(columns={'Populacao':'POPULAÇÃO RESIDENTE',
                                'CAIXAS':'NÚMERO DE CAIXAS',
                                'TESTES':'NÚMERO DE TESTES PREVISTOS',
                                'Municipio':'MUNICÍPIO'})

testes_municipios.to_excel(writer,'por_municipio', index=False)

In [ ]:
testes_rs = tabela_base.groupby('RS')[['Populacao', 'CAIXAS', 'TESTES']].sum().reset_index()
testes_rs.loc['TOTAL', ['Populacao', 'CAIXAS', 'TESTES']] = testes_rs[['Populacao', 'CAIXAS', 'TESTES']].sum()
testes_rs

In [ ]:
# POR RS

In [ ]:
testes_rs = testes_rs.rename(columns={'Populacao':'POPULAÇÃO RESIDENTE',
                                'CAIXAS':'NÚMERO DE CAIXAS',
                                'TESTES':'NÚMERO DE TESTES PREVISTOS'})
testes_rs.to_excel(writer,'por_rs', index=False)

In [ ]:
# POR MACRO

In [ ]:
testes_macro = tabela_base.groupby('MACRO')[['Populacao', 'CAIXAS', 'TESTES']].sum().reset_index()
testes_macro.loc['TOTAL', ['Populacao', 'CAIXAS', 'TESTES']] = testes_macro[['Populacao', 'CAIXAS', 'TESTES']].sum()
testes_macro

In [ ]:
testes_macro = testes_macro.rename(columns={'Populacao':'POPULAÇÃO RESIDENTE',
                                'CAIXAS':'NÚMERO DE CAIXAS',
                                'TESTES':'NÚMERO DE TESTES PREVISTOS'})
testes_macro.to_excel(writer,'por_macro', index=False)

In [ ]:
writer.save()
writer.close()